In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# FraudFinder - BigQuery ML - Model training and prediction

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/fraudfinder/raw/main/bqml/04_model_training_and_prediction.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/bqml/04_model_training_and_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/bqml/04_model_training_and_prediction.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[FraudFinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the FraudFinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

After feature engineering and ingesting data into a feature store, you are now ready to train and deploy your machine learning model to predict whether a transaction is fraudulent or not.

In this notebook, using the data in Vertex AI Feature Store that you previously ingested data into, you will train a model using BigQuery ML, register the model to Vertex AI Model Registry, and deploy it to an endpoint for real-time prediction. 

In this tutorial, you will learn how to:

- Train a logistic regression model in BigQuery using BigQuery ML
- Evaluate the model
- Test a prediction 
- Deploy to an endpoint on Vertex AI
- Make an online prediction

This tutorial uses the following Google Cloud data analytics and services:

- [BigQuery](https://cloud.google.com/bigquery/)
- [BigQuery ML](https://cloud.google.com/bigquery-ml/)
- [Vertex AI](https://cloud.google.com/vertex-ai/)

The steps performed include:

- Using Python & SQL to query the data in BigQuery
- Preparing the data for modeling
- Training a classification model using BigQuery ML and registering it to Vertex AI Model Registry
- Inspecting the model on Vertex AI Model Registry
- Deploying the model to an endpoint on Vertex AI
- Making sample online predictions to the model endpoint

### Costs 

This tutorial uses billable components of Google Cloud:

* BigQuery
* BigQuery ML
* Vertex AI

Learn about [BigQuery Pricing](https://cloud.google.com/bigquery/pricing), [BigQuery ML pricing](https://cloud.google.com/bigquery-ml/pricing), [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [1]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)


BUCKET_NAME          = "fraud123-438914-fraudfinder"
PROJECT              = "fraud123-438914"
REGION               = "us-central1"
ID                   = "fvde2"
FEATURESTORE_ID      = "fraudfinder_fvde2"
MODEL_NAME           = "ff_model"
ENDPOINT_NAME        = "ff_model_endpoint"
TRAINING_DS_SIZE     = "1000"



### Import libraries

In [2]:
import pandas as pd
import json

from datetime import datetime, timedelta

from typing import Union

from google.cloud import aiplatform as vertex_ai
from google.cloud import bigquery

pd.set_option("display.max_columns", None)

### Define constants

In [3]:
START_DATE_TRAIN = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")
END_DATE_TRAIN = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")
CUSTOMERS_TABLE_NAME = f"customers_{END_DATE_TRAIN.replace('-', '')}"
TERMINALS_TABLE_NAME = f"terminals_{END_DATE_TRAIN.replace('-', '')}"
VERTEX_AI_MODEL_ID = 'bqml_fraud_classifier'
SERVING_FEATURE_IDS = {"customer": ["*"], "terminal": ["*"]}
READ_INSTANCES_TABLE = f"ground_truth_{END_DATE_TRAIN.replace('-', '')}"
READ_INSTANCES_URI = f"bq://{PROJECT_ID}.tx.{READ_INSTANCES_TABLE}"
BQ_TABLE_NAME = f"train_table_{END_DATE_TRAIN.replace('-', '')}"
TRAIN_TABLE_URI = f"bq://{PROJECT_ID}.tx.{BQ_TABLE_NAME}"

print(f"""
START_DATE_TRAIN \t= {START_DATE_TRAIN}
END_DATE_TRAIN \t\t= {END_DATE_TRAIN}
CUSTOMERS_TABLE_NAME \t= {CUSTOMERS_TABLE_NAME}
TERMINALS_TABLE_NAME \t= {TERMINALS_TABLE_NAME}
VERTEX_AI_MODEL_ID \t= {VERTEX_AI_MODEL_ID}
SERVING_FEATURE_IDS \t= {SERVING_FEATURE_IDS}
READ_INSTANCES_TABLE \t= {READ_INSTANCES_TABLE}
READ_INSTANCES_URI \t= {READ_INSTANCES_URI}
BQ_TABLE_NAME \t\t= {BQ_TABLE_NAME}
TRAIN_TABLE_URI \t= {TRAIN_TABLE_URI}
""")


START_DATE_TRAIN 	= 2024-10-16
END_DATE_TRAIN 		= 2024-10-16
CUSTOMERS_TABLE_NAME 	= customers_20241016
TERMINALS_TABLE_NAME 	= terminals_20241016
VERTEX_AI_MODEL_ID 	= bqml_fraud_classifier
SERVING_FEATURE_IDS 	= {'customer': ['*'], 'terminal': ['*']}
READ_INSTANCES_TABLE 	= ground_truth_20241016
READ_INSTANCES_URI 	= bq://fraud123-438914.tx.ground_truth_20241016
BQ_TABLE_NAME 		= train_table_20241016
TRAIN_TABLE_URI 	= bq://fraud123-438914.tx.train_table_20241016



#### Payload schema

The following schema will be used later in this notebook for prediction.

In [4]:
PAYLOAD_SCHEMA = {
    "tx_amount": "float64",
    "customer_id_nb_tx_1day_window": "int64",
    "customer_id_nb_tx_7day_window": "int64",
    "customer_id_nb_tx_14day_window": "int64",
    "customer_id_avg_amount_1day_window": "float64",
    "customer_id_avg_amount_7day_window": "float64",
    "customer_id_avg_amount_14day_window": "float64",
    "customer_id_nb_tx_15min_window": "int64",
    "customer_id_avg_amount_15min_window": "float64",
    "customer_id_nb_tx_30min_window": "int64",
    "customer_id_avg_amount_30min_window": "float64",
    "customer_id_nb_tx_60min_window": "int64",
    "customer_id_avg_amount_60min_window": "float64",
    "terminal_id_nb_tx_1day_window": "int64",
    "terminal_id_nb_tx_7day_window": "int64",
    "terminal_id_nb_tx_14day_window": "int64",
    "terminal_id_risk_1day_window": "float64",
    "terminal_id_risk_7day_window": "float64",
    "terminal_id_risk_14day_window": "float64",
    "terminal_id_nb_tx_15min_window": "int64",
    "terminal_id_avg_amount_15min_window": "float64",
    "terminal_id_nb_tx_30min_window": "int64",
    "terminal_id_avg_amount_30min_window": "float64",
    "terminal_id_nb_tx_60min_window": "int64",
    "terminal_id_avg_amount_60min_window": "float64",
}

### Initialize Vertex AI and BigQuery SDKs for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [5]:
vertex_ai.init(project=PROJECT_ID, location=REGION)

### Helpers

Use a helper function for sending queries to BigQuery.

In [6]:
# Wrapper to use BigQuery client to run query/job, return job ID or result as DF
def run_bq_query(sql: str) -> Union[str, pd.DataFrame]:
    """
    Run a BigQuery query and return the job ID or result as a DataFrame
    Args:
        sql: SQL query, as a string, to execute in BigQuery
    Returns:
        df: DataFrame of results from query,  or error, if any
    """

    bq_client = bigquery.Client()

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

## Fetching feature values for model training

In the previous labs `02_feature_engineering_batch.ipynb` and `03_feature_engineering_streaming`, you computed features and ingested them into Vertex AI Feature Store. You can now look back into Feature Store to any point in time and output those values from Feature Store in batch to a BigQuery table in order to train your machine learning model.

Note that in these Fraudfinder labs, the ground-truth fraud labels are already available for all transactions, even as soon as a new transaction occurs. However, in practice, there is usually a delay (in th order of _days_) between when a transaction occurs until it gets officially flagged as a fraudulent transaction by a fraud department. Because of this, if the data you use to train on doesn't yet have the up-to-date information about whether it's truly fraud or not, you may end up training an ML model on incorrect data (i.e. on transactions that are actually fraud, but were labeled as not-fraud at the time of training). If this is the case, then when you choose what data you use for training a model, be sure you use the point-in-time by "time-traveling" an appropriate number of days into the past to a date when you're confident that the labels should be accurate when fetching feature values from Vertex AI Feature Store. For this notebook, you will simply be "time traveling" to yesterday for simplicity.

To fetch data from Vertex AI Feature Store (via batch serving), you will need to provide the following inputs:

- a file containing a "query", with the entities and timestamps for each transaction
- a list of features to fetch values for
- the destination location and format where you want your feature values to be outputted to


### Preparing the data to query Vertex AI Feature Store (the "Read-Instance")

You will first need to prepare data that you want to use to query Vertex AI Feature Store. It will need to be a CSV file formatted like the table below:

|customer                     |terminal|timestamp                                    |
|-----------------------------|--------|---------------------------------------------|
|xxx3859                         |xxx8811    |2021-07-07 00:01:10 UTC                      |
|xxx4165                         |xxx8810    |2021-07-07 00:01:55 UTC                      |
|xxx2289                         |xxx2081    |2021-07-07 00:02:12 UTC                      |
|xxx3227                         |xxx3011    |2021-07-07 00:03:23 UTC                      |
|xxx2819                         |xxx6263    |2021-07-07 00:05:30 UTC                      |

where the column names are the name of entities in Feature Store and the timestamps represents the time an event occurred. Vertex AI Feature Store will then retrieve the last feature value up to (but not after) those timestamps.

In [7]:
read_instances_query = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.tx.{READ_INSTANCES_TABLE}` as (
    SELECT
        raw_tx.TX_TS AS timestamp,
        raw_tx.CUSTOMER_ID AS customer,
        raw_tx.TERMINAL_ID AS terminal,
        raw_tx.TX_AMOUNT AS tx_amount,
        raw_lb.TX_FRAUD AS tx_fraud,
    FROM 
        tx.tx as raw_tx
    LEFT JOIN 
        tx.txlabels as raw_lb
    ON raw_tx.TX_ID = raw_lb.TX_ID
    WHERE
        DATE(raw_tx.TX_TS) = "{START_DATE_TRAIN}"
);
"""
print(read_instances_query)

run_bq_query(read_instances_query)
run_bq_query(f"SELECT * FROM `{PROJECT_ID}.tx.{READ_INSTANCES_TABLE}` LIMIT 10")


CREATE OR REPLACE TABLE `fraud123-438914.tx.ground_truth_20241016` as (
    SELECT
        raw_tx.TX_TS AS timestamp,
        raw_tx.CUSTOMER_ID AS customer,
        raw_tx.TERMINAL_ID AS terminal,
        raw_tx.TX_AMOUNT AS tx_amount,
        raw_lb.TX_FRAUD AS tx_fraud,
    FROM 
        tx.tx as raw_tx
    LEFT JOIN 
        tx.txlabels as raw_lb
    ON raw_tx.TX_ID = raw_lb.TX_ID
    WHERE
        DATE(raw_tx.TX_TS) = "2024-10-16"
);

Finished job_id: 26439b7f-4b97-4b34-86f4-0b1fc7dde810
Finished job_id: 9cb4bb2c-3a67-4eeb-829c-ba9fb778afbf


,timestamp,customer,terminal,tx_amount,tx_fraud
0,2024-10-16 08:37:16+00:00,4896317891393653,00064542,78.700000000,0
1,2024-10-16 11:42:59+00:00,4176271867313310,00064542,82.440000000,0
2,2024-10-16 16:04:46+00:00,2753236058497187,00064542,10.110000000,0
3,2024-10-16 16:28:15+00:00,7596157283232196,00064542,36.510000000,0
4,2024-10-16 05:14:50+00:00,6177866302228204,00064542,23.830000000,0
5,2024-10-16 17:20:26+00:00,4377145325468797,00064542,45.590000000,0
6,2024-10-16 01:09:01+00:00,9034584161813968,00064542,14.870000000,0
7,2024-10-16 03:09:07+00:00,7655238087830723,00064542,34.720000000,0
8,2024-10-16 08:22:15+00:00,8284238040012214,00064542,42.590000000,0
9,2024-10-16 02:18:23+00:00,6580876822931629,00064542,70.190000000,0


### Initiate Feature Store

Initiate the feature store you created in the `02_feature_engineering_batch.ipynb` notebook.

In [8]:
try:
    ff_feature_store = vertex_ai.Featurestore(f"{FEATURESTORE_ID}")
    print(f"""The feature store {FEATURESTORE_ID} found!""")
except NameError:
    print(f"""The feature store {FEATURESTORE_ID} does not exist!""")

The feature store fraudfinder_fvde2 found!


### Export a sample of data to a BigQuery using Vertex AI Feature Store's point-in-time capability

In this section, we will use batch serving of feature store to prepare a dataset for training by calling the `BatchReadFeatureValues` API. Batch serving is used to fetch a large set of feature values with high throughput, typically for training a model or batch prediction. For outputting the results to BigQuery, you will use the [`batch_serve_to_bq` method](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Featurestore#google_cloud_aiplatform_Featurestore_batch_serve_to_bq).

In the follow cell, based on the table you created above, you will query feature store for all of the relevant feature values, and export the feature values into a new table in BigQuery, which can the be used for model training later on.

In [9]:
ff_feature_store.batch_serve_to_bq(
    bq_destination_output_uri=TRAIN_TABLE_URI,
    serving_feature_ids=SERVING_FEATURE_IDS,
    read_instances_uri=READ_INSTANCES_URI,
    pass_through_fields=["tx_amount", "tx_fraud"],
)

print(f"Feature values from feature store outputted to: {TRAIN_TABLE_URI}.")

Serving Featurestore feature values: projects/520607199607/locations/us-central1/featurestores/fraudfinder_fvde2
Serve Featurestore feature values backing LRO: projects/520607199607/locations/us-central1/featurestores/fraudfinder_fvde2/operations/7352050147422896128
Featurestore feature values served. Resource name: projects/520607199607/locations/us-central1/featurestores/fraudfinder_fvde2
Feature values from feature store outputted to: bq://fraud123-438914.tx.train_table_20241016.


### Inspect the resulting output table from Vertex AI Feature Store

Run the query below to inspect the training table. 

In [10]:
sql_inspect = f"""
SELECT
    *
FROM
    `tx.{BQ_TABLE_NAME}`
LIMIT
    5
"""
run_bq_query(sql_inspect)

Finished job_id: 94dd81c5-bb5a-49bb-88a8-5062054da5a0


,tx_amount,tx_fraud,timestamp,entity_type_customer,customer_id_avg_amount_15min_window,customer_id_nb_tx_7day_window,customer_id_avg_amount_14day_window,customer_id_avg_amount_60min_window,customer_id_avg_amount_1day_window,customer_id_nb_tx_30min_window,customer_id_avg_amount_30min_window,customer_id_nb_tx_14day_window,customer_id_nb_tx_15min_window,customer_id_avg_amount_7day_window,customer_id_nb_tx_1day_window,customer_id_nb_tx_60min_window,entity_type_terminal,terminal_id_nb_tx_30min_window,terminal_id_risk_1day_window,terminal_id_avg_amount_60min_window,terminal_id_nb_tx_7day_window,terminal_id_avg_amount_15min_window,terminal_id_risk_14day_window,terminal_id_nb_tx_1day_window,terminal_id_nb_tx_60min_window,terminal_id_risk_7day_window,terminal_id_nb_tx_14day_window,terminal_id_avg_amount_30min_window,terminal_id_nb_tx_15min_window
0,90.870000000,0,2024-10-16 22:21:47+00:00,8563281253156795,0.0,25,81.366444,0.0,82.632000,0,0.0,45,0,82.034000,5,0,80054181,0,0.0,0.0,44,0.0,0.0,4,0,0.0,63,0.0,0
1,4.100000000,0,2024-10-16 15:47:27+00:00,7926124324804711,0.0,20,6.220976,0.0,4.745000,0,0.0,41,0,5.650000,2,0,80054181,0,0.0,0.0,44,0.0,0.0,4,0,0.0,61,0.0,0
2,16.060000000,0,2024-10-16 22:00:28+00:00,4263864643438713,0.0,28,16.930962,0.0,16.628571,0,0.0,52,0,17.500000,7,0,80054181,0,0.0,0.0,44,0.0,0.0,4,0,0.0,63,0.0,0
3,3.540000000,0,2024-10-16 15:38:25+00:00,4882328591972575,0.0,23,6.926596,0.0,5.385000,0,0.0,47,0,6.505217,4,0,80054181,0,0.0,0.0,45,0.0,0.0,4,0,0.0,61,0.0,0
4,6.190000000,0,2024-10-16 21:08:21+00:00,7926124324804711,0.0,20,6.289302,0.0,6.217500,0,0.0,43,0,5.878000,4,0,80054181,0,0.0,0.0,45,0.0,0.0,4,0,0.0,63,0.0,0


As you can see, each row represents a transaction ID, and the columns represent the attributes of the transaction (i.e `customer_id`, `terminal_id`, `tx_amount`),  aggregated features (for the customer and terminal) and the ground-truth label `tx_fraud` (1 if fraud, else 0).

## Using BigQuery ML to train a machine learning model directly in BigQuery

BigQuery ML (BQML) provides the capability to train machine learning models, such as classification, regression, forecasting, and matrix factorization, in BigQuery using SQL syntax directly. BigQuery ML uses the scalable infrastructure of BigQuery so you don't need to set up additional infrastructure for training or batch serving.

Because the data from Vertex AI Feature Store was outputted to a BigQuery table, this makes the data conveniently available for training directly using BigQuery ML.

Learn more about [BigQuery ML documentation](https://cloud.google.com/bigquery-ml/docs).

### Train a logistic regression model using BigQuery ML

The query below trains a logistic regression model using BigQuery ML. BigQuery resources are used to train the model.

In the `OPTIONS` parameter:
* with `model_registry="vertex_ai"`, the BigQuery ML model will automatically be [registered to Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/model-registry-bqml), which enables you to view all of your registered models and its versions on Google Cloud in one place.

* `vertex_ai_model_version_aliases` allows you to set aliases to help you keep track of your model version ([documentation](https://cloud.google.com/vertex-ai/docs/model-registry/model-alias)).

In [11]:
# this cell may take ~4 min to run
sql_train_model_bqml = f"""
CREATE OR REPLACE MODEL `tx.{MODEL_NAME}` 
OPTIONS(
  MODEL_TYPE="LOGISTIC_REG",
  INPUT_LABEL_COLS=["tx_fraud"],
  EARLY_STOP=TRUE,
  MIN_REL_PROGRESS=0.01,
  model_registry="vertex_ai", 
  vertex_ai_model_id='{VERTEX_AI_MODEL_ID}',
  vertex_ai_model_version_aliases=['logit', 'experimental']
) AS

SELECT
  * EXCEPT(timestamp, entity_type_customer, entity_type_terminal)
FROM
   `tx.{BQ_TABLE_NAME}`
"""

print(sql_train_model_bqml)

run_bq_query(sql_train_model_bqml)

print(f"Training job finished for: `tx.{MODEL_NAME}`")


CREATE OR REPLACE MODEL `tx.ff_model` 
OPTIONS(
  MODEL_TYPE="LOGISTIC_REG",
  INPUT_LABEL_COLS=["tx_fraud"],
  EARLY_STOP=TRUE,
  MIN_REL_PROGRESS=0.01,
  model_registry="vertex_ai", 
  vertex_ai_model_id='bqml_fraud_classifier',
  vertex_ai_model_version_aliases=['logit', 'experimental']
) AS

SELECT
  * EXCEPT(timestamp, entity_type_customer, entity_type_terminal)
FROM
   `tx.train_table_20241016`

Finished job_id: a616376f-a15f-4625-b2b3-5a3525be65a0
Training job finished for: `tx.ff_model`


#### Inspect the model on Vertex AI Model Registry
The model should now be automatically registered to Vertex AI Model Registry upon completion.

You can view the model on the <a href="https://console.cloud.google.com/vertex-ai/models" target="_blank">Vertex AI Model Registry page</a>, or use the code below to check that it was successfully registered:

In [12]:
# Initiate Vertex AI Model Registry for `VERTEX_AI_MODEL_ID` model entry
registry = vertex_ai.models.ModelRegistry(VERTEX_AI_MODEL_ID)

In [13]:
# Get model versions
versions = registry.list_versions()

for version in versions:
    version_id = version.version_id
    version_created_time = datetime.fromtimestamp(
        version.version_create_time.timestamp()
    ).strftime("%m/%d/%Y %H:%M:%S")
    version_aliases = version.version_aliases
    print(
        f"Model version {version_id} was created at {version_created_time} with aliases {version_aliases}",
    )

Getting versions for projects/fraud123-438914/locations/us-central1/models/bqml_fraud_classifier
Model version 1 was created at 10/17/2024 17:40:16 with aliases ['logit', 'experimental', 'default']


In [14]:
# Get the model
model = registry.get_model(version="logit")
print(model)

resource name: projects/520607199607/locations/us-central1/models/bqml_fraud_classifier


### Model evaluation

With the model created, you can now evaluate the logistic regression model. Behind the scenes, BigQuery ML automatically [split the data](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create#data_split_method), which makes it easier to quickly train and evaluate models.

In [15]:
sql_evaluate_model = f"""
SELECT
  *
FROM
  ML.EVALUATE(MODEL tx.{MODEL_NAME})
"""

print(sql_evaluate_model)

run_bq_query(sql_evaluate_model)


SELECT
  *
FROM
  ML.EVALUATE(MODEL tx.ff_model)

Finished job_id: 680e576a-5d7f-4fa3-87f4-1fab9ca079ba


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,1.0,0.828221,0.997219,0.90604,0.015203,0.979266


These metrics help you understand the performance of the model. 

There are various metrics for logistic regression and other model types (full list of metrics can be found in the [documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate#mlevaluate_output)).

### Batch prediction (with Explainable AI)

Make a batch prediction in BigQuery ML on the original training data to check the probability of a transaction to be fraudulent for transaction, as seen in the `probability` column, with the predicted label under the `predicted_tx_fraud` column.

[ML.EXPLAIN_PREDICT](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-explain-predict) has built-in [Explainable AI](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-xai-overview). This allows you to see the top contributing features to each prediction and interpret how it was computed.

In [16]:
sql_explain_predict = f"""
SELECT
  *
FROM
  ML.EXPLAIN_PREDICT(MODEL tx.{MODEL_NAME},
    (SELECT * FROM  `tx.{BQ_TABLE_NAME}` LIMIT 10)
    )
"""

print(sql_explain_predict)

run_bq_query(sql_explain_predict)


SELECT
  *
FROM
  ML.EXPLAIN_PREDICT(MODEL tx.ff_model,
    (SELECT * FROM  `tx.train_table_20241016` LIMIT 10)
    )

Finished job_id: 8935ec06-ddff-46b5-bc39-8c1c328660ba


,predicted_tx_fraud,probability,top_feature_attributions,baseline_prediction_value,prediction_value,approximation_error,tx_amount,tx_fraud,timestamp,entity_type_customer,customer_id_avg_amount_15min_window,customer_id_nb_tx_7day_window,customer_id_avg_amount_14day_window,customer_id_avg_amount_60min_window,customer_id_avg_amount_1day_window,customer_id_nb_tx_30min_window,customer_id_avg_amount_30min_window,customer_id_nb_tx_14day_window,customer_id_nb_tx_15min_window,customer_id_avg_amount_7day_window,customer_id_nb_tx_1day_window,customer_id_nb_tx_60min_window,entity_type_terminal,terminal_id_nb_tx_30min_window,terminal_id_risk_1day_window,terminal_id_avg_amount_60min_window,terminal_id_nb_tx_7day_window,terminal_id_avg_amount_15min_window,terminal_id_risk_14day_window,terminal_id_nb_tx_1day_window,terminal_id_nb_tx_60min_window,terminal_id_risk_7day_window,terminal_id_nb_tx_14day_window,terminal_id_avg_amount_30min_window,terminal_id_nb_tx_15min_window
0,0,0.988154,"[{'feature': 'tx_amount', 'attribution': 1.051...",-5.715449,-4.423807,0.0,90.870000000,0,2024-10-16 22:21:47+00:00,8563281253156795,0.0,25,81.366444,0.0,82.632000,0,0.0,45,0,82.034000,5,0,80054181,0,0.0,0.0,44,0.0,0.0,4,0,0.0,63,0.0,0
1,0,0.999661,"[{'feature': 'tx_amount', 'attribution': -1.56...",-5.715449,-7.989420,0.0,4.100000000,0,2024-10-16 15:47:27+00:00,7926124324804711,0.0,20,6.220976,0.0,4.745000,0,0.0,41,0,5.650000,2,0,80054181,0,0.0,0.0,44,0.0,0.0,4,0,0.0,61,0.0,0
2,0,0.997611,"[{'feature': 'tx_amount', 'attribution': -1.20...",-5.715449,-6.034425,0.0,16.060000000,0,2024-10-16 22:00:28+00:00,4263864643438713,0.0,28,16.930962,0.0,16.628571,0,0.0,52,0,17.500000,7,0,80054181,0,0.0,0.0,44,0.0,0.0,4,0,0.0,63,0.0,0
3,0,0.999357,"[{'feature': 'tx_amount', 'attribution': -1.57...",-5.715449,-7.347959,0.0,3.540000000,0,2024-10-16 15:38:25+00:00,4882328591972575,0.0,23,6.926596,0.0,5.385000,0,0.0,47,0,6.505217,4,0,80054181,0,0.0,0.0,45,0.0,0.0,4,0,0.0,61,0.0,0
4,0,0.999439,"[{'feature': 'tx_amount', 'attribution': -1.49...",-5.715449,-7.484498,0.0,6.190000000,0,2024-10-16 21:08:21+00:00,7926124324804711,0.0,20,6.289302,0.0,6.217500,0,0.0,43,0,5.878000,4,0,80054181,0,0.0,0.0,45,0.0,0.0,4,0,0.0,63,0.0,0
5,0,0.992095,"[{'feature': 'tx_amount', 'attribution': 1.067...",-5.715449,-4.832287,0.0,91.380000000,0,2024-10-16 14:55:33+00:00,4747071652870389,0.0,24,82.814444,0.0,84.103333,0,0.0,45,0,84.670000,3,0,80054181,0,0.0,0.0,45,0.0,0.0,4,0,0.0,61,0.0,0
6,0,0.994454,"[{'feature': 'tx_amount', 'attribution': 0.773...",-5.715449,-5.189059,0.0,81.630000000,0,2024-10-16 18:37:15+00:00,8563281253156795,0.0,24,81.280000,0.0,79.416667,0,0.0,44,0,81.421667,3,0,80054181,0,0.0,0.0,46,0.0,0.0,4,0,0.0,63,0.0,0
7,0,0.999567,"[{'feature': 'customer_id_nb_tx_1day_window', ...",-5.715449,-7.744084,0.0,57.710000000,0,2024-10-16 05:34:08+00:00,8624547480913904,0.0,9,69.182222,0.0,57.710000,0,0.0,18,0,67.008889,1,0,80054181,0,0.0,0.0,48,0.0,0.0,6,0,0.0,60,0.0,0
8,0,0.995785,"[{'feature': 'tx_amount', 'attribution': 0.809...",-5.715449,-5.464976,0.0,82.830000000,0,2024-10-16 05:39:07+00:00,7695898795468133,0.0,19,89.283659,0.0,86.543333,0,0.0,41,0,89.175263,3,0,80054181,0,0.0,0.0,48,0.0,0.0,6,0,0.0,60,0.0,0
9,0,0.999251,"[{'feature': 'tx_amount', 'attribution': 0.597...",-5.715449,-7.195960,0.0,75.780000000,0,2024-10-16 08:31:47+00:00,7356529610529253,0.0,8,80.158750,0.0,75.780000,0,0.0,16,0,79.773750,1,0,80054181,0,0.0,0.0,48,0.0,0.0,7,0,0.0,61,0.0,0


Since the `top_feature_attributions` is a nested column, you can unnest the array ([documentation](https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays)) into separate rows for each of the features. In other words, since ML.EXPLAIN_PREDICT provides the top 5 most important features, using `UNNEST` results in 5 rows per prediction:

In [17]:
sql_explain_predict = f"""
SELECT
  tfa.*,
  predicted_tx_fraud,
  probability,
  baseline_prediction_value,
  prediction_value,
  approximation_error,
FROM
  ML.EXPLAIN_PREDICT(MODEL tx.{MODEL_NAME},
    (SELECT * FROM `tx.{BQ_TABLE_NAME}` )
    ),
  UNNEST(top_feature_attributions) as tfa
LIMIT 100
"""

print(sql_explain_predict)

run_bq_query(sql_explain_predict)


SELECT
  tfa.*,
  predicted_tx_fraud,
  probability,
  baseline_prediction_value,
  prediction_value,
  approximation_error,
FROM
  ML.EXPLAIN_PREDICT(MODEL tx.ff_model,
    (SELECT * FROM `tx.train_table_20241016` )
    ),
  UNNEST(top_feature_attributions) as tfa
LIMIT 100

Finished job_id: 1fabb16a-9e35-40e9-8597-bbe93359739f


,feature,attribution,predicted_tx_fraud,probability,baseline_prediction_value,prediction_value,approximation_error
0,tx_amount,1.051718,0,0.988154,-5.715449,-4.423807,0.0
1,customer_id_avg_amount_1day_window,0.353316,0,0.988154,-5.715449,-4.423807,0.0
2,customer_id_avg_amount_14day_window,-0.298697,0,0.988154,-5.715449,-4.423807,0.0
3,customer_id_nb_tx_7day_window,0.219343,0,0.988154,-5.715449,-4.423807,0.0
4,customer_id_nb_tx_1day_window,0.213853,0,0.988154,-5.715449,-4.423807,0.0
...,...,...,...,...,...,...,...
95,tx_amount,1.363897,0,0.965777,-5.715449,-3.340028,0.0
96,customer_id_avg_amount_1day_window,0.517300,0,0.965777,-5.715449,-3.340028,0.0
97,customer_id_nb_tx_7day_window,0.497709,0,0.965777,-5.715449,-3.340028,0.0
98,customer_id_avg_amount_14day_window,-0.459211,0,0.965777,-5.715449,-3.340028,0.0


### Deploy the model to an endpoint

While BigQuery ML supports batch prediction with [ML.PREDICT](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict) and [ML.EXPLAIN_PREDICT](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-explain-predict), BigQuery ML is not suitable for real-time predictions where you need low latency predictions with potentially high frequency of requests.

In other words, deploying the BigQuery ML model to an endpoint enables you to do online predictions.

#### Create a Vertex AI endpoint

To deploy your model to an endpoint, you will first need to create an endpoint before you deploy the model to it.

In [18]:
endpoint = vertex_ai.Endpoint.create(
    display_name=ENDPOINT_NAME,
    project=PROJECT_ID,
    location=REGION,
)

print(endpoint.display_name)
print(endpoint.resource_name)

Creating Endpoint
Create Endpoint backing LRO: projects/520607199607/locations/us-central1/endpoints/6298724983040376832/operations/3919744281413156864
Endpoint created. Resource name: projects/520607199607/locations/us-central1/endpoints/6298724983040376832
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/520607199607/locations/us-central1/endpoints/6298724983040376832')
ff_model_endpoint
projects/520607199607/locations/us-central1/endpoints/6298724983040376832


#### List endpoints

List the endpoints to make sure it has successfully been created. You can also view your endpoints on the [Vertex AI Endpoints page](https://console.cloud.google.com/vertex-ai/endpoints).

In [19]:
endpoint.list(order_by="update_time")

 resource name: projects/520607199607/locations/us-central1/endpoints/6298724983040376832]

#### Deploy model to Vertex endpoint

With the model, you can now deploy it to an endpoint. 

In [20]:
# deploying the model to the endpoint may take 10-15 minutes
model.deploy(endpoint=endpoint)

Deploying model to Endpoint : projects/520607199607/locations/us-central1/endpoints/6298724983040376832
Using default machine_type: n1-standard-2
Deploy Endpoint model backing LRO: projects/520607199607/locations/us-central1/endpoints/6298724983040376832/operations/3800961841241260032


InvalidArgument: 400 Error occurred in Explanation preprocessing. <class 'ValueError'> NodeDef mentions attr 'debug_name' not in Op<name=VarHandleOp; signature= -> resource:resource; attr=container:string,default=""; attr=shared_name:string,default=""; attr=dtype:type; attr=shape:shape; is_stateful=true>; NodeDef: {{node weights}}. (Check whether your GraphDef-interpreting binary is up to date with your GraphDef-generating binary.).
 3: Error occurred in Explanation preprocessing. <class 'ValueError'> NodeDef mentions attr 'debug_name' not in Op<name=VarHandleOp; signature= -> resource:resource; attr=container:string,default=""; attr=shared_name:string,default=""; attr=dtype:type; attr=shape:shape; is_stateful=true>; NodeDef: {{node weights}}. (Check whether your GraphDef-interpreting binary is up to date with your GraphDef-generating binary.).


You can also check on the status of your model by visiting the [Vertex AI Endpoints page](https://console.cloud.google.com/vertex-ai/endpoints).

### Make online predictions to the endpoint with pub/sub -> pull subscription -> notebook -> Vertex AI endpoint
Using a sample of the training data, you can test the endpoint to make online predictions.

Below are some helper functions to help make it easier to read streaming data and do online predictions:

In [21]:
# A function to read some sample messages (transaction) from the public Pub/Sub
def read_from_sub(project_id, subscription_path, messages=10):
    """
    Read messages from a Pub/Sub subscription
    Args:
        project_id: project ID
        subscription_name: the name of a Pub/Sub subscription in your project
        messages: number of messages to read
    Returns:
        msg_data: list of messages in your Pub/Sub subscription as a Python dictionary
    """
    import ast

    from google.api_core import retry
    from google.cloud import pubsub_v1

    subscriber = pubsub_v1.SubscriberClient()
    subscription_path = subscriber.subscription_path(project_id, subscription_path)

    # Wrap the subscriber in a 'with' block to automatically call close() to
    # close the underlying gRPC channel when done.
    with subscriber:
        # The subscriber pulls a specific number of messages. The actual
        # number of messages pulled may be smaller than max_messages.
        response = subscriber.pull(
            subscription=subscription_path,
            max_messages=messages,
            retry=retry.Retry(deadline=300),
        )

        if len(response.received_messages) == 0:
            print("no messages")
            return

        ack_ids = []
        msg_data = []
        for received_message in response.received_messages:
            msg = ast.literal_eval(received_message.message.data.decode("utf-8"))
            print(f"Received: {msg}.")
            msg_data.append(msg)
            ack_ids.append(received_message.ack_id)

        # Acknowledges the received messages so they will not be sent again.
        subscriber.acknowledge(subscription=subscription_path, ack_ids=ack_ids)

        print(
            f"Received and acknowledged {len(response.received_messages)} messages from {subscription_path}."
        )

        return msg_data


# A function for pre-processing of payload before sending it to a Vertex AI endpoint
def preprocess(payload):
    """
    Preprocesses the payload before sending it to a Vertex AI endpoint
    Args:
        payload: payload to be preprocessed
    Returns:
        payload: preprocessed payload
    """
    # replace NaN's
    for key, value in payload.items():
        if value is None:
            payload[key] = 0.0
    return payload


# A function to lookup features in Vertex AI Feature Store
def features_lookup(ff_feature_store, entity, entity_ids):
    """
    Looks up features in Vertex AI Feature Store
    Args:
        ff_feature_store: Feature Store object
        entity: entity type
        entity_ids: list of entity IDs
    Returns:
        features: list of features
    """
    entity_type = ff_feature_store.get_entity_type(entity)
    aggregated_features = entity_type.read(entity_ids=entity_ids, feature_ids="*")
    aggregated_features_preprocessed = preprocess(aggregated_features)
    features = aggregated_features_preprocessed.iloc[0].to_dict()
    return features

You can now read some messages from Pub/Sub, preprocess it, augment it with some features from Vertex AI Feature Store, and submit to Vertex AI Endpoint for online predictions.

In [22]:
messages = read_from_sub(
    project_id=PROJECT_ID, subscription_path="ff-tx-sub", messages=10
)

for payload_input in messages:
    print(f"----The raw transaction from Pub/Sub----")
    print(json.dumps(payload_input, indent=4))
    print(f"----The input payload to the Vertex AI endpoint:----")
    
    payload = {}
    payload["tx_amount"] = payload_input["TX_AMOUNT"]
    
    # look up the customer features from feature store (written by batch ingestion daily and by Dataflow in real-time)
    customer_features = features_lookup(
        ff_feature_store, "customer", [payload_input["CUSTOMER_ID"]]
    )
    payload["customer_id_nb_tx_1day_window"] = customer_features["customer_id_nb_tx_1day_window"]
    payload["customer_id_nb_tx_7day_window"] = customer_features["customer_id_nb_tx_7day_window"]
    payload["customer_id_nb_tx_14day_window"] = customer_features["customer_id_nb_tx_14day_window"]
    payload["customer_id_avg_amount_1day_window"] = customer_features["customer_id_avg_amount_1day_window"]
    payload["customer_id_avg_amount_7day_window"] = customer_features["customer_id_avg_amount_7day_window"]
    payload["customer_id_avg_amount_14day_window"] = customer_features["customer_id_avg_amount_14day_window"]
    payload["customer_id_nb_tx_15min_window"] = customer_features["customer_id_nb_tx_15min_window"]
    payload["customer_id_avg_amount_15min_window"] = customer_features["customer_id_avg_amount_15min_window"]
    payload["customer_id_nb_tx_30min_window"] = customer_features["customer_id_nb_tx_30min_window"]
    payload["customer_id_avg_amount_30min_window"] = customer_features["customer_id_avg_amount_30min_window"]
    payload["customer_id_nb_tx_60min_window"] = customer_features["customer_id_nb_tx_60min_window"]
    payload["customer_id_avg_amount_60min_window"] = customer_features["customer_id_avg_amount_60min_window"]

    # look up the terminal features from feature store (written by batch ingestion daily and by Dataflow in real-time)
    terminal_features = features_lookup(
        ff_feature_store, "terminal", [payload_input["TERMINAL_ID"]]
    )
    
    payload["terminal_id_nb_tx_1day_window"] = terminal_features["terminal_id_nb_tx_1day_window"]
    payload["terminal_id_nb_tx_7day_window"] = terminal_features["terminal_id_nb_tx_7day_window"]
    payload["terminal_id_nb_tx_14day_window"] = terminal_features["terminal_id_nb_tx_14day_window"]
    payload["terminal_id_risk_1day_window"] = terminal_features["terminal_id_risk_1day_window"]
    payload["terminal_id_risk_7day_window"] = terminal_features["terminal_id_risk_7day_window"]
    payload["terminal_id_risk_14day_window"] = terminal_features["terminal_id_risk_14day_window"]
    payload["terminal_id_nb_tx_15min_window"] = terminal_features["terminal_id_nb_tx_15min_window"]
    payload["terminal_id_avg_amount_15min_window"] = terminal_features["terminal_id_avg_amount_15min_window"]
    payload["terminal_id_nb_tx_30min_window"] = terminal_features["terminal_id_nb_tx_30min_window"]
    payload["terminal_id_avg_amount_30min_window"] = terminal_features["terminal_id_avg_amount_30min_window"]
    payload["terminal_id_nb_tx_60min_window"] = terminal_features["terminal_id_nb_tx_60min_window"]
    payload["terminal_id_avg_amount_60min_window"] = terminal_features["terminal_id_avg_amount_60min_window"]
    payload = preprocess(payload)
    
    print(json.dumps(payload, indent=4))
    print(f"----The prediction result:----")

    result = endpoint.predict(instances=[payload])
    print(json.dumps(result, indent=4))
    print(f"===============================================================")

Received: {'TX_ID': '875ce3cf02c4f5d6e95112ce02a40eabe134a73f', 'TX_TS': '2024-10-17 18:22:04', 'CUSTOMER_ID': '5394657612275705', 'TERMINAL_ID': '73505401', 'TX_AMOUNT': 15.19}.
Received: {'TX_ID': 'faa33b0cec36e9d8c83e4faa05664a1395c4773c', 'TX_TS': '2024-10-17 18:13:20', 'CUSTOMER_ID': '0559586904113034', 'TERMINAL_ID': '83999652', 'TX_AMOUNT': 5.02}.
Received: {'TX_ID': '3302bcfc3fa82fff6b90a788415d9d1e563a957a', 'TX_TS': '2024-10-17 18:17:20', 'CUSTOMER_ID': '4758383396097798', 'TERMINAL_ID': '47302377', 'TX_AMOUNT': 16.44}.
Received: {'TX_ID': '146ccec8ec0d086550124c7853d2da7f89963e49', 'TX_TS': '2024-10-17 18:17:28', 'CUSTOMER_ID': '6507590528282358', 'TERMINAL_ID': '68149143', 'TX_AMOUNT': 68.73}.
Received: {'TX_ID': 'b00834a7ea72cb7d6a842c0dcb965e042fc824f9', 'TX_TS': '2024-10-17 18:19:19', 'CUSTOMER_ID': '9766845828256737', 'TERMINAL_ID': '05509090', 'TX_AMOUNT': 30.02}.
Received: {'TX_ID': 'cded3f3949abbc46c6ca6fb8d87e89353d024ce4', 'TX_TS': '2024-10-17 18:19:27', 'CUSTOMER_

### END

Now you can go to the next notebook `05_model_training_pipeline_formalization.ipynb`

### Clean up

In [ ]:
# endpoint[-1].undeploy_all()

In [ ]:
# delete_model_sql = f"""
# DROP MODEL `{PROJECT_ID}.{BQ_DATASET}.{BQML_MODEL_NAME}`
# """

# bq_query(delete_model_sql)